# Watershed Delineation
The notebook shows how to perform a watershed delineation with `quest` including downloading elevation data, hydrologically preparing the dataset, and finally extracting watersheds for provided outlet points. This notebook uses additional visualization and input tools from [Pyviz](http://pyviz.org).

## setup

In addition to `quest` the following packages will also need to be installed in the environment to run this notebook:
* holoviews
* geoviews
* parambokeh

These packages can be installed with the following command:
```
conda install -c pyviz/label/dev/ holoviews geoviews parambokeh
```

In [ ]:
import parambokeh
import quest
import geoviews as gv
import holoviews as hv
from holoviews.streams import (PolyEdit, BoxEdit, PointDraw)
from holoviews.operation.datashader import regrid

regrid.aggregator = 'max'
hv.extension('bokeh')
%output holomap='scrubber' fps=2

## Draw bounds to compute watershed
Allows drawing a bounding box and adding points to serve as input to compute a watershed:

In [ ]:
tiles = gv.tile_sources.StamenTerrain().options(width=950, height=600)
tiles.extents=(-125, 25, -65, 50)
box_poly = gv.Polygons([]).options(fill_alpha=.2)
box_stream = BoxEdit(source=box_poly, num_objects=1)
tiles * box_poly

In [ ]:
if box_stream.element:
    xs, ys = box_stream.element.array().T
    bbox = [xs[0], ys[1], xs[2], ys[0]]
else:
    bbox = [-90.88870324076336, 32.245105881134, -90.78133198716839, 32.37688211930573]
    
print('BOUNDS:', bbox)

In [ ]:
elevation_service = quest.util.ServiceSelector(parameter='elevation', default='svc://usgs-ned:1-arc-second')
parambokeh.Widgets(elevation_service)

In [ ]:
elevation_raster = quest.api.get_seamless_data(
    service_uri=elevation_service.service,
    bbox=bbox,
    collection_name='examples',
    use_cache=True,
    as_open_dataset=False,
)

In [ ]:
fill_dataset = quest.tools.wbt_fill_depressions(dataset=elevation_raster)['datasets'][0]
fill = quest.api.open_dataset(fill_dataset, with_nodata=True, isel_band=0)

In [ ]:
options = quest.tools.wbt_extract_streams_workflow
options.dataset = fill_dataset
options.params()['dataset'].precedence = -1
options.set_threshold_bounds()
parambokeh.Widgets(options, next_n=2)

In [ ]:
st_dataset = options()['datasets'][0]
st = quest.api.open_dataset(st_dataset, with_nodata=True, isel_band=0)

In [ ]:
%%opts Points [width=900 height=600] (size=10 color='red')
tiles = gv.tile_sources.StamenTerrain.options(width=950, height=600)
tiles.extents=tuple(bbox)
points = gv.Points([])
point_stream = PointDraw(source=points)
elevation = gv.Image(fill, ['x', 'y']).options(alpha=0.8)
streams = gv.Image(st, ['x', 'y']).options(cmap='blues_r')
tiles * regrid(elevation) * regrid(streams) * points

In [ ]:
if point_stream.element:
    original_outlets = [(x, y) for x, y in zip(*point_stream.element.array().T)]
else:
    original_outlets = (-90.883981967599979, 32.291221825861946)
    
print('OUTLETS:', original_outlets)

In [ ]:
result = quest.tools.wbt_watershed_delineation_workflow(
    elevation_dataset=fill_dataset,
    streams_dataset=st_dataset,
    snap_distance=0.1,
    outlets=original_outlets,
)
watersheds, snapped_outlets, catalog_entry = result['catalog_entries']

In [ ]:
tiles = gv.tile_sources.StamenTerrain().options(width=950, height=600)
tiles = gv.tile_sources.StamenTerrain().options(width=950, height=600)
tiles.extents=tuple(bbox)
outline = gv.Polygons(watersheds).options(alpha=0.5)
original_points = gv.Points(original_outlets).options(color='blue', size=6)
snapped_points = gv.Points(snapped_outlets).options(color='red', size=6)
tiles * regrid(elevation) * outline * regrid(streams) * original_points * snapped_points